In [40]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Reshape
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv3D, MaxPooling3D, LSTM, TimeDistributed
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [41]:
with open('train_20_1908_04092019_only_image.pickle', 'rb') as f:
    X_train, y_train = pickle.load(f)

In [42]:
X_train = X_train[..., np.newaxis]
print(X_train.shape,y_train.shape)

(1908, 20, 50, 50, 1) (1908,)


In [43]:
Y_train = keras.utils.to_categorical(y_train,14)

In [44]:
print(X_train.shape,y_train.shape)

(1908, 20, 50, 50, 1) (1908,)


In [45]:
X_train,X_dev,Y_train,Y_dev = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True)
print(X_train.shape)

(1717, 20, 50, 50, 1)


In [46]:
model = Sequential()
model.add(Conv3D(20, kernel_size=(3, 3, 50),
                 activation='relu',
                 input_shape=(20,50,50,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Conv3D(50, (2, 3, 20), activation='relu',padding = 'same'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Conv3D(100, (3, 3, 50), activation='relu', padding = 'same'))
model.add(MaxPooling3D(pool_size=(2, 2, 1))) 
model.add(Conv3D(100, (3, 3, 100), activation='relu', padding = 'same'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Dropout(0.2))
#model.add(Reshape((2*2*1,100)))
model.add(Flatten()) 
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
#print_weights = keras.callbacks.LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()))

#model.add(LSTM(100, activation='tanh', return_sequences=False))
model.add(Dense(14, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [47]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 18, 48, 1, 20)     9020      
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 9, 24, 1, 20)      0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 9, 24, 1, 50)      120050    
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 4, 12, 1, 50)      0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 4, 12, 1, 100)     2250100   
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 2, 6, 1, 100)      0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 2, 6, 1, 100)     

In [48]:
filepath="image_only_weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(X_train, Y_train,
          batch_size=200,
          epochs=200,
          verbose=1,
          callbacks=callbacks_list,
#           steps_per_epoch = 3,
          validation_data=(X_dev, Y_dev))
model.save('image_only_tasnim+07')
model.save_weights('image_only_W_2007.h5')

Train on 1717 samples, validate on 191 samples
Epoch 1/200
1600/1717 [==========================>...] - ETA: 0s - loss: 2.6392 - acc: 0.0725
Epoch 00001: val_acc improved from -inf to 0.08377, saving model to image_only_weights-improvement-01-0.08.h5
1717/1717 [==============================] - 12s 7ms/sample - loss: 2.6393 - acc: 0.0705 - val_loss: 2.6399 - val_acc: 0.0838
Epoch 2/200
1600/1717 [==========================>...] - ETA: 0s - loss: 2.6388 - acc: 0.0719
Epoch 00002: val_acc did not improve from 0.08377
1717/1717 [==============================] - 11s 6ms/sample - loss: 2.6386 - acc: 0.0734 - val_loss: 2.6401 - val_acc: 0.0838
Epoch 3/200
1600/1717 [==========================>...] - ETA: 0s - loss: 2.6383 - acc: 0.0719
Epoch 00003: val_acc did not improve from 0.08377
1717/1717 [==============================] - 11s 6ms/sample - loss: 2.6382 - acc: 0.0734 - val_loss: 2.6412 - val_acc: 0.0838
Epoch 4/200
1600/1717 [==========================>...] - ETA: 0s - loss: 2.6381 - a

1717/1717 [==============================] - 18s 11ms/sample - loss: 2.5678 - acc: 0.1136 - val_loss: 2.5740 - val_acc: 0.1099
Epoch 31/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.5468 - acc: 0.1306
Epoch 00031: val_acc improved from 0.10995 to 0.14136, saving model to image_only_weights-improvement-31-0.14.h5
1717/1717 [==============================] - 19s 11ms/sample - loss: 2.5491 - acc: 0.1305 - val_loss: 2.5560 - val_acc: 0.1414
Epoch 32/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.5363 - acc: 0.1306
Epoch 00032: val_acc improved from 0.14136 to 0.15183, saving model to image_only_weights-improvement-32-0.15.h5
1717/1717 [==============================] - 19s 11ms/sample - loss: 2.5328 - acc: 0.1334 - val_loss: 2.5333 - val_acc: 0.1518
Epoch 33/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.5083 - acc: 0.1612
Epoch 00033: val_acc improved from 0.15183 to 0.16754, saving model to image_only_weights-improvement-33-0.1

1600/1717 [==========================>...] - ETA: 1s - loss: 2.1422 - acc: 0.2713
Epoch 00059: val_acc did not improve from 0.25654
1717/1717 [==============================] - 18s 10ms/sample - loss: 2.1473 - acc: 0.2697 - val_loss: 2.2825 - val_acc: 0.2094
Epoch 60/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.1553 - acc: 0.2612
Epoch 00060: val_acc did not improve from 0.25654
1717/1717 [==============================] - 18s 11ms/sample - loss: 2.1625 - acc: 0.2592 - val_loss: 2.2629 - val_acc: 0.2147
Epoch 61/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.1294 - acc: 0.2788
Epoch 00061: val_acc did not improve from 0.25654
1717/1717 [==============================] - 18s 11ms/sample - loss: 2.1272 - acc: 0.2819 - val_loss: 2.2422 - val_acc: 0.2304
Epoch 62/200
1600/1717 [==========================>...] - ETA: 1s - loss: 2.1040 - acc: 0.2925
Epoch 00062: val_acc did not improve from 0.25654
1717/1717 [==============================] - 18s 10m

1717/1717 [==============================] - 19s 11ms/sample - loss: 1.9127 - acc: 0.3465 - val_loss: 2.1678 - val_acc: 0.2880
Epoch 89/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.8952 - acc: 0.3619
Epoch 00089: val_acc did not improve from 0.28796
1717/1717 [==============================] - 19s 11ms/sample - loss: 1.8928 - acc: 0.3599 - val_loss: 2.1975 - val_acc: 0.2827
Epoch 90/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.8963 - acc: 0.3562
Epoch 00090: val_acc did not improve from 0.28796
1717/1717 [==============================] - 19s 11ms/sample - loss: 1.9015 - acc: 0.3535 - val_loss: 2.1668 - val_acc: 0.2775
Epoch 91/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.8980 - acc: 0.3613
Epoch 00091: val_acc did not improve from 0.28796
1717/1717 [==============================] - 18s 11ms/sample - loss: 1.8998 - acc: 0.3623 - val_loss: 2.1412 - val_acc: 0.2723
Epoch 92/200
1600/1717 [==========================>...] -

1600/1717 [==========================>...] - ETA: 1s - loss: 1.7637 - acc: 0.3950
Epoch 00118: val_acc did not improve from 0.31414
1717/1717 [==============================] - 19s 11ms/sample - loss: 1.7745 - acc: 0.3943 - val_loss: 2.1031 - val_acc: 0.3037
Epoch 119/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.7583 - acc: 0.4112
Epoch 00119: val_acc did not improve from 0.31414
1717/1717 [==============================] - 18s 11ms/sample - loss: 1.7606 - acc: 0.4100 - val_loss: 2.1456 - val_acc: 0.2775
Epoch 120/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.7809 - acc: 0.3938
Epoch 00120: val_acc did not improve from 0.31414
1717/1717 [==============================] - 19s 11ms/sample - loss: 1.7678 - acc: 0.3978 - val_loss: 2.1390 - val_acc: 0.3089
Epoch 121/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.7345 - acc: 0.4087
Epoch 00121: val_acc did not improve from 0.31414
1717/1717 [==============================] - 19s 

Epoch 148/200
1600/1717 [==========================>...] - ETA: 0s - loss: 1.5710 - acc: 0.4644
Epoch 00148: val_acc did not improve from 0.33508
1717/1717 [==============================] - 15s 9ms/sample - loss: 1.5752 - acc: 0.4630 - val_loss: 2.1633 - val_acc: 0.3141
Epoch 149/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.5526 - acc: 0.4800
Epoch 00149: val_acc did not improve from 0.33508
1717/1717 [==============================] - 16s 10ms/sample - loss: 1.5499 - acc: 0.4776 - val_loss: 2.1818 - val_acc: 0.3037
Epoch 150/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.5654 - acc: 0.4663
Epoch 00150: val_acc did not improve from 0.33508
1717/1717 [==============================] - 18s 10ms/sample - loss: 1.5654 - acc: 0.4630 - val_loss: 2.2056 - val_acc: 0.3037
Epoch 151/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.5504 - acc: 0.4613
Epoch 00151: val_acc did not improve from 0.33508
1717/1717 [=========================

1600/1717 [==========================>...] - ETA: 1s - loss: 1.4237 - acc: 0.5256
Epoch 00178: val_acc did not improve from 0.33508
1717/1717 [==============================] - 18s 10ms/sample - loss: 1.4257 - acc: 0.5213 - val_loss: 2.2398 - val_acc: 0.3037
Epoch 179/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.4184 - acc: 0.5188
Epoch 00179: val_acc did not improve from 0.33508
1717/1717 [==============================] - 18s 11ms/sample - loss: 1.4253 - acc: 0.5178 - val_loss: 2.2148 - val_acc: 0.3194
Epoch 180/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.3932 - acc: 0.5362
Epoch 00180: val_acc improved from 0.33508 to 0.35079, saving model to image_only_weights-improvement-180-0.35.h5
1717/1717 [==============================] - 19s 11ms/sample - loss: 1.3954 - acc: 0.5358 - val_loss: 2.1783 - val_acc: 0.3508
Epoch 181/200
1600/1717 [==========================>...] - ETA: 1s - loss: 1.3935 - acc: 0.5175
Epoch 00181: val_acc did not improv